<a href="https://colab.research.google.com/github/jwengr/KoDeBERTa/blob/main/lit_deberta_colab_tpu_pretrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
torch.__version__

'2.0.1+cu118'

In [2]:
!pip install --quiet cloud-tpu-client==0.10 torch==2.0.0 torchvision==0.15.1 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.9/162.9 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 106.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━

In [3]:
! pip install --quiet pytorch-lightning transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.6/720.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 91.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [13]:
cd /content/drive/MyDrive/KoDeBERTa

/content/drive/MyDrive/KoDeBERTa
/content/drive/MyDrive/KoDeBERTa


In [ ]:
!git clone https://github.com/jwengr/KoDeBERTa.git

In [2]:
import torch
import pytorch_lightning as pl

from tokenizers import Tokenizer
from datasets import IterableDataset
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint

from KoDeBERTa.Data.DataCollator import DataCollatorForHFUnigramSpanMLM
from KoDeBERTa.Model.DebertaV3 import LitDebertaV3ForPretraining

In [3]:
SEED=0

In [4]:
pl.seed_everything(SEED)

INFO:lightning_fabric.utilities.seed:Global seed set to 0


0

In [5]:
tokenizer = Tokenizer.from_file('./tokenizers/mecab-hf-unigram-880M-128k.json')

In [6]:
mask_id = tokenizer.get_vocab()['[MASK]']
pad_id = tokenizer.get_vocab()['[PAD]']

In [7]:
def gen():
    with open("/content/drive/MyDrive/KoDeBERTa/data/mecab.txt", encoding='utf-8-sig') as f:
        for line in f:
            yield line

In [8]:
batch_size = 16
max_steps = 500_000 * (8192//batch_size)

In [9]:
ds = IterableDataset.from_generator(gen)
ds.shuffle(seed=SEED, buffer_size=8_800_000)
dl = DataLoader(ds, batch_size=batch_size, collate_fn=DataCollatorForHFUnigramSpanMLM(tokenizer, truncation_argument={'max_length':512}))

In [10]:
debertav3_pretrainer = LitDebertaV3ForPretraining('microsoft/deberta-v3-xsmall', mask_id=mask_id, pad_id=pad_id, lr=1e-7, num_warmup_steps=int(max_steps*0.08), num_training_steps=max_steps)

In [14]:
checkpoint_callback = ModelCheckpoint(
    monitor='Loss_D',
    dirpath='./model/tpu',
    filename='{epoch:02d}-{Loss_G:.4f}-{Loss_D:.4f}',
    every_n_train_steps=500_000//20
)

trainer = pl.Trainer(
    accelerator='gpu',
    # devices=1,
    precision='bf16-mixed',
    max_steps=max_steps,
    callbacks=[checkpoint_callback],
)


MisconfigurationException: ignored

In [12]:
trainer.fit(debertav3_pretrainer,dl)

INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type                            | Params
------------------------------------------------------------------
0 | generator     | DebertaV2ForMaskedLM            | 60.3 M
1 | discriminator | DebertaV2ForTokenClassification | 70.7 M
------------------------------------------------------------------
130 M     Trainable params
0         Non-trainable params
130 M     Total params
523.980   Total estimated model params size (MB)
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
